The purpose of the project is to learn the mapping from polar coordinates to a a discrete 10x10 grid of cells in the plane, using a neural network. 

The supervised dataset is given to you in the form of a generator (to be considered as a black box).

The model must achieve an accuracy of 95%, and it will be evaluated in a way **inversely proportional to the number of its parameters: the smaller, the better.**

**WARNING**: Any solution taking advantage of meta-knowledge about the generator will be automatically rejected.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Concatenate
from tensorflow.keras.models import Model

Here is the generator. It returns triples of the form ((theta,rho),out) where (theta,rho) are the polar coordinates of a point in the first quadrant of the plane, and out is a 10x10 map with "1" in the cell corresponding to the point position, and "0" everywhere else.

By setting flat=True, the resulting map is flattened into a vector with a single dimension 100. You can use this variant, if you wish. 

In [2]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=True):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

Let's create an instance of the generator on a grid with dimension 3x4

In [3]:
g1,g2 = 3,4
gen = polar_generator(1,grid=(g1,g2),noise=0.0)

And now let's see a few samples.

In [4]:
(theta,rho),maps = next(gen)
for i,map in enumerate(maps):
  #let us compute the cartesian coordinates
  x = np.cos(theta[i])*rho[i]
  y = np.sin(theta[i])*rho[i]
  print("x coordinate (row): {}".format(int(x*g1)))
  print("y coordinate (col): {}".format(int(y*g2)))
  print("map:")
  print(np.reshape(map,(g1,g2)))
  #and the polar coordinates
  print("polar coordinates of this point are:")
  print(theta[i], rho[i])

x coordinate (row): 0
y coordinate (col): 2
map:
[[0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
polar coordinates of this point are:
1.2585603491769635 0.5568679232331946


Exercise: add noise to the generator, and check the effect on the "ground truth".

# What to deliver

For the purposes of the project you are supposed to work with the **default 10x10 grid, and the default noise=.002**

The generator must be treatead as a black box, do not tweak it, and do not exploit its semantics that is supposed to be unknown. You are allowed to work with the "flat" modality, if you prefer so.

You need to:
1.   define an accuracy function (take inspiration from the code of the previous cell)
2.   define a neural network taking in input theta and rho, and returning out
3. measure the network's accuracy that must be above 95% (accuracy must be evaluated over at least 20000 samples)
4. tune the network trying to decrease as much as possible the numer of parameters, preserving an accuracy above 95%. Only your best network must be delivered.

You must deliver a SINGLE notebook working on colab, containing the code of the network, its summary, the training history, the code for the accurary metric and its evaluation on the network.

**N.B.** The accuracy must be above 95% but apart from that it does not influence the evaluation. You score will only depend on the number of parameters: the lower, the better.

#Good work!





#Delivery

##Exercise

Let's first solve the firs exercise: adding noise to the generator, then compare the result with "ground truth".

In [5]:
noise_generator = polar_generator(1,grid=(g1,g2),noise=0.002)
(n_theta,n_rho),n_maps = next(noise_generator)
print((n_theta,n_rho),n_maps)
print()

# now let's compute the cartesian coordinates from 
# the polar ones given from the noised generator
x = np.cos(n_theta[0])*n_rho[0]
y = np.sin(n_theta[0])*n_rho[0]
print("cartesian coordinates for theta: {} and rho: {} are".format(n_theta[0],n_rho[0]))
print("x: {} and y: {}".format(int(x*g1),int(y*g2)))
print()

# compute the polar coordinates from the cartesian ones
# given from the noised generator
for map in n_maps:
  for i,el in enumerate(map):
    if int(el) == 1:
      pos = i
xc = int(pos/4)
yc = int(pos%4)
x = x/g1
y = y/g2
theta_i = np.arctan(y/np.maximum(x,.00001))
rho_i = np.sqrt(x**2+y**2)
print("polar coordinates for x: {} and y: {} are".format(xc,yc))
print("theta: {} and rho: {}".format(theta_i,rho_i))

(array([0.69448818]), array([1.09230336])) [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]

cartesian coordinates for theta: 0.6944881788200966 and rho: 1.0923033617131261 are
x: 2 and y: 2

polar coordinates for x: 2 and y: 2 are
theta: 0.558370776919886 and rho: 0.32986925350225643


We can observe that when adding noise to the generator, values for both theta and rho are slighlty different from the one we actually get if computing angle and radius given cartesian coordinates.

##Accuracy function

First we set our global parameters, then we define the accuracy function.

In [6]:
# set dimension of the grid where to map polar coordinates
g1,g2 = 10,10

# set numnber of both training and testing samples
training_samples = 5000
testing_samples = 20000

def accuracy(model, samples_no, generator):

  # generate our samples
  (thetas, rhos), true_output = next(generator(samples_no))

  # map to grid by using the model
  predicted_output = model.predict([np.array(thetas), np.array(rhos)])

  score = 0
  for i in range(samples_no):

    # calculate cartesian coordinates from input data
    x = np.cos(thetas[i])*rhos[i]
    y = np.sin(thetas[i])*rhos[i]
    xc1 = int(x*g1)
    yc1 = int(y*g2)

    # get cartesian coordinates from predicted data
    pos = predicted_output[i].argmax()
    xc2 = int(pos/g1)
    yc2 = int(pos%g2)
    if (xc1 == xc2) and (yc1 == yc2):
      score += 1
  #print("accuracy given by model.evaluate() is: {}".format(model.evaluate([thetas, rhos], true_output)))
  yield (score/samples_no)

##Neural Network

Let's define the network. The idea behind it is to divide the first quadrant of the cartesian plane in sub-qadrants, verifying, depending on theta and rho values, where to discriminate the point position.

In [7]:
# input layers, one for theta and one for rho
x1in = Input(shape=(1,))
x2in = Input(shape=(1,))

xin = Concatenate(axis=1)([x1in, x2in])

# dense layers that mimic sub-quadrants of the first cartesian quadrant
x1 = Dense(1, activation='relu')(xin)
x2 = Dense(1, activation='relu')(xin)
x3 = Dense(1, activation='relu')(xin)
x4 = Dense(1, activation='relu')(xin)
x5 = Dense(1, activation='relu')(xin)

y = Concatenate(axis=1)([x1,x2,x3,x4,x5])

z = Dense(20,activation='relu')(y)

mylayer = Dense(100,activation='softmax')(z)

mynet = Model(inputs=[x1in,x2in],outputs=mylayer)
mynet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 2)            0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 1)            3           ['concatenate[0][0]']        

It is time to train the network.

In [8]:
mynet.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

mynet.fit(polar_generator(training_samples), steps_per_epoch=1000, epochs=20)

Epoch 1/20
1000/1000 [==============================] - 29s 23ms/step - loss: 3.0182 - accuracy: 0.3175
Epoch 2/20
1000/1000 [==============================] - 19s 19ms/step - loss: 1.3073 - accuracy: 0.8005
Epoch 3/20
1000/1000 [==============================] - 19s 19ms/step - loss: 0.8245 - accuracy: 0.8564
Epoch 4/20
1000/1000 [==============================] - 20s 20ms/step - loss: 0.6234 - accuracy: 0.8721
Epoch 5/20
1000/1000 [==============================] - 20s 20ms/step - loss: 0.5152 - accuracy: 0.8825
Epoch 6/20
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4468 - accuracy: 0.8905
Epoch 7/20
1000/1000 [==============================] - 19s 19ms/step - loss: 0.4007 - accuracy: 0.8967
Epoch 8/20
1000/1000 [==============================] - 19s 19ms/step - loss: 0.3665 - accuracy: 0.9024
Epoch 9/20
1000/1000 [==============================] - 19s 19ms/step - loss: 0.3394 - accuracy: 0.9082
Epoch 10/20
1000/1000 [==============================] - 20s 20m

Now we measure our model accuracy

In [9]:
print(next(accuracy(mynet, testing_samples, polar_generator)))

625/625 [==============================] - 1s 2ms/step
0.9479
